### IMPORTATION DES BIBLIOTHEQUES

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import dash
import threading
import webbrowser
from dash import Dash, dcc, html, Input, Output
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut
import time
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report
import joblib  # Pour sauvegarder le modèle
from fastapi import FastAPI
import joblib
import pandas as pd
import threading
import uvicorn
from pydantic import BaseModel
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight


%matplotlib inline

### IMPORTATION ET NETTOYAGE DE LA BASE

In [ ]:
Base=pd.read_excel("Updated Challenge dataset.xlsx")
Base.index=Base.index+1
Base['Arrondissement de résidence'] = Base['Arrondissement de résidence'].str.replace("Douala (Non précisé )", "DOUALA", regex=False)
Base['Arrondissement de résidence'] = Base['Arrondissement de résidence'].str.replace("Yaounde", "Yaoundé", regex=False)
Base = Base[~Base['Arrondissement de résidence'].isin(["Pas précisé", "Pas mentionné", 'ras ','rien',"R A S", "RAS", "Pas precise" ,"R A S "," R A S ",' R A S','Pas precisé','Pas précisé ','Ras','non precisé','Non précisé','Aucun','Non précisé',"Pas precise",'non precisé',"Pas précisé ","Ras", "R A S ","R A S",'RAS '])]
valeurs_a_remplacer = ["Pas précisé", "Pas mentionné",'ras ','rien', "R A S", "RAS", "Pas precise" ,"R A S "," R A S ",' R A S','Pas precisé','Pas précisé ','Ras','non precisé','Non précisé','Aucun','Non précisé',"Pas precise",'non precisé',"Pas précisé ","Ras", "R A S ","R A S","r .a.s"]

# Appliquer la condition
Base['Quartier de Résidence'] = Base.apply(
    lambda row: row['Arrondissement de résidence'] if row['Quartier de Résidence'] in valeurs_a_remplacer else row['Quartier de Résidence'],
    axis=1
)
Base['Arrondissement de résidence']=Base['Arrondissement de résidence'].str.upper()
Base['Quartier de Résidence']=Base['Quartier de Résidence'].str.lower()

valeurs_a_remplacer1 = ["new bel", "new bell", "newbell"]

# Appliquer la condition
Base['Quartier de Résidence'] = Base.apply(
    lambda row: "new-bell" if row['Quartier de Résidence'] in valeurs_a_remplacer1 else row['Quartier de Résidence'],
    axis=1
)

valeurs_a_remplacer2 = ["bp cite", "bp site"]

# Appliquer la condition
Base['Quartier de Résidence'] = Base.apply(
    lambda row: "bp cité" if row['Quartier de Résidence'] in valeurs_a_remplacer2 else row['Quartier de Résidence'],
    axis=1
)

valeurs_a_remplacer3 = ["brassaville", "brazaville"]

# Appliquer la condition
Base['Quartier de Résidence'] = Base.apply(
    lambda row: "brazzaville" if row['Quartier de Résidence'] in valeurs_a_remplacer3 else row['Quartier de Résidence'],
    axis=1
)
valeurs_a_remplacer5 = ["cite"]

# Appliquer la condition
Base['Quartier de Résidence'] = Base.apply(
    lambda row: "cité" if row['Quartier de Résidence'] in valeurs_a_remplacer5 else row['Quartier de Résidence'],
    axis=1
)
valeurs_a_remplacer4 = ["cité cic"]

# Appliquer la condition
Base['Quartier de Résidence'] = Base.apply(
    lambda row: "sic" if row['Quartier de Résidence'] in valeurs_a_remplacer4 else row['Quartier de Résidence'],
    axis=1
)

In [ ]:
def corriger_quartier(quatier_str):
    if isinstance(quatier_str,str) and "log" in quatier_str and "baba" in quatier_str:
        quatier_str="logbaba"
    if isinstance(quatier_str,str) and "log" in quatier_str and "bes" in quatier_str and "ou" in quatier_str:
        quatier_str="logbessou"
    if isinstance(quatier_str,str) and "nd" in quatier_str and "g" in quatier_str and ("bon" in quatier_str or "bom" in quatier_str):
        quatier_str="ndogbong"
    if isinstance(quatier_str,str) and ("do" in quatier_str or "da" in quatier_str) and ("pass" in quatier_str or "bass" in quatier_str or "poss" in quatier_str) and ("3" in quatier_str or "iii" in quatier_str or "march" in quatier_str):
        quatier_str="ndogpassi"+" 3"
    if isinstance(quatier_str,str) and ("do" in quatier_str or "da" in quatier_str) and ("pass" in quatier_str or "bass" in quatier_str or "poss" in quatier_str) and ("2" in quatier_str or "ii" in quatier_str):
        quatier_str="ndogpassi"+" 2"
    if isinstance(quatier_str,str) and ("do" in quatier_str or "da" in quatier_str) and ("pass" in quatier_str or "bass" in quatier_str or "poss" in quatier_str) and not ("2" in quatier_str or "3" in quatier_str or "ii" in quatier_str or "iii" in quatier_str):
        quatier_str="ndogpassi"+" 1"
    if isinstance(quatier_str,str) and "bep" in quatier_str and ("enda" in quatier_str or "anda" in quatier_str):
        quatier_str="bepanda"
    if isinstance(quatier_str,str) and "ari" in quatier_str:
        quatier_str="ari village"
    if isinstance(quatier_str,str) and ("makepe" in quatier_str or "makèpè" in quatier_str):
        quatier_str="makepe"
    if isinstance(quatier_str,str) and "beedi" in quatier_str:
        quatier_str="beedi"
    if isinstance(quatier_str,str) and "mous" in quatier_str and "sadi" in quatier_str and not "sable" in quatier_str:
        quatier_str="bonamoussadi"
    if isinstance(quatier_str,str) and "nialla" in quatier_str:
        quatier_str=quatier_str.replace("nialla","nyalla")
    if isinstance(quatier_str,str) and "ngalla" in quatier_str:
        quatier_str=quatier_str.replace("ngalla","nyalla")
    if isinstance(quatier_str,str) and "nyala" in quatier_str:
        quatier_str=quatier_str.replace("nyala","nyalla")
    if isinstance(quatier_str,str) and "nyassa" in quatier_str:
        quatier_str=quatier_str.replace("nyassa","yassa")
    if isinstance(quatier_str,str) and ("she’ll" in quatier_str or "shele" in quatier_str) :
        quatier_str="shell village"
    if isinstance(quatier_str,str) and ("cité" in quatier_str or "cite" in quatier_str) and ("sic" in quatier_str or "cic" in quatier_str) or quatier_str=='sic':
        quatier_str="cité sic"
    if isinstance(quatier_str,str) and "deido " in quatier_str:
        quatier_str=quatier_str.replace("deido ","deido")
    if isinstance(quatier_str,str) and "déido" in quatier_str:
        quatier_str=quatier_str.replace("déido","deido")
    if isinstance(quatier_str,str) and "deïdo" in quatier_str:
        quatier_str=quatier_str.replace("deïdo","deido")
    if isinstance(quatier_str,str) and "diedo" in quatier_str:
        quatier_str=quatier_str.replace("diedo","deido")
    if isinstance(quatier_str,str) and "rafael" in quatier_str:
        quatier_str="ange raphaël"
    if isinstance(quatier_str,str) and "japouma" in quatier_str:
        quatier_str=quatier_str.replace("japouma","japoma")
    if isinstance(quatier_str,str) and "japoma " in quatier_str:
        quatier_str=quatier_str.replace("japoma ","japoma")
    if isinstance(quatier_str,str) and "besengue" in quatier_str:
        quatier_str=quatier_str.replace("besengue","bessengue")
    if isinstance(quatier_str,str) and "bonaberie" in quatier_str:
        quatier_str=quatier_str.replace("bonaberie","bonaberi")
    if isinstance(quatier_str,str) and "bonabéri" in quatier_str:
        quatier_str=quatier_str.replace("bonabéri","bonaberi")
    if isinstance(quatier_str,str) and quatier_str=="bonabéri ":
        quatier_str="bonaberi"
    if isinstance(quatier_str,str) and quatier_str=="bonaberi ":
        quatier_str="bonaberi"
    if isinstance(quatier_str,str) and "bonaberi" in quatier_str:
        quatier_str=quatier_str.replace("bonaberi","bonaberi")
    if isinstance(quatier_str,str) and "bonaorisso" in quatier_str:
        quatier_str=quatier_str.replace("bonaorisso","bonapriso")
    if isinstance(quatier_str,str) and "bonandjo" in quatier_str:
        quatier_str=quatier_str.replace("bonandjo","bonanjo")
    if isinstance(quatier_str,str) and "bonandjo" in quatier_str:
        quatier_str=quatier_str.replace("bonandjo","bonanjo")
    if isinstance(quatier_str,str) and "villoage" in quatier_str:
        quatier_str=quatier_str.replace("villoage","village")
    if isinstance(quatier_str,str) and "villoage" in quatier_str:
        quatier_str=quatier_str.replace("villoage","village")
    if isinstance(quatier_str,str) and "akwa-nord" in quatier_str:
        quatier_str=quatier_str.replace("akwa-nord","akwa nord")
    if isinstance(quatier_str,str) and "akwa-nord" in quatier_str:
        quatier_str=quatier_str.replace("akwa-nord","akwa nord")
    if isinstance(quatier_str,str) and "cit" in quatier_str and "pa" in quatier_str and "mier" in quatier_str:
        quatier_str="cité des palmiers"
    if isinstance(quatier_str,str) and "boko-plage" in quatier_str:
        quatier_str=quatier_str.replace("boko-plage","boko plage")
    if isinstance(quatier_str,str) and "pl12" in quatier_str:
        quatier_str=quatier_str.replace("pl12","pk12")
    if isinstance(quatier_str,str) and "pk 11" in quatier_str:
        quatier_str=quatier_str.replace("pk 11","pk11")
    if isinstance(quatier_str,str) and "pk 9" in quatier_str:
        quatier_str=quatier_str.replace("pk 9","pk9")
    if isinstance(quatier_str,str) and "pk9 " in quatier_str:
        quatier_str=quatier_str.replace("pk9 ","pk9")
    if isinstance(quatier_str,str) and quatier_str=="logpom ":
        quatier_str="logpom"
    if isinstance(quatier_str,str) and quatier_str=="yassa ":
        quatier_str="yassa"
    if isinstance(quatier_str,str) and quatier_str=="akoua":
        quatier_str="akwa"
    if isinstance(quatier_str,str) and quatier_str=="ndokotti " or quatier_str=="ndokoti " or quatier_str=="ndokotti":
        quatier_str="ndokoti"
    if isinstance(quatier_str,str) and "pital" in quatier_str:
        quatier_str="hôpital général"
    if isinstance(quatier_str,str) and "nko" in quatier_str and ("ondo" in quatier_str or "odo" in quatier_str):
        quatier_str="nkomondo"
    if isinstance(quatier_str,str) and "ccc" in quatier_str:
        quatier_str="ccc"
    if isinstance(quatier_str,str) and "borne" in quatier_str and "10" in quatier_str:
        quatier_str="village borne 10"
    if isinstance(quatier_str,str) and "pk" in quatier_str and "16" in quatier_str:
        quatier_str="pk16"
    if isinstance(quatier_str,str) and "pk" in quatier_str and "5" in quatier_str:
        quatier_str="pk5"
    if isinstance(quatier_str,str) and "pk" in quatier_str and "8" in quatier_str:
        quatier_str="pk8"
    if isinstance(quatier_str,str) and "pk" in quatier_str and "10" in quatier_str:
        quatier_str="pk10"
    if isinstance(quatier_str,str) and "pk" in quatier_str and "15" in quatier_str:
        quatier_str="pk15"
    if isinstance(quatier_str,str) and "pk" in quatier_str and "18" in quatier_str:
        quatier_str="pk18"
    if isinstance(quatier_str,str) and "pk" in quatier_str and "23" in quatier_str:
        quatier_str="pk23"
    if isinstance(quatier_str,str) and quatier_str=="édea" or quatier_str=="edea " or quatier_str=="edéa ":
        quatier_str="edea"
    if isinstance(quatier_str,str) and "marche" in quatier_str:
        quatier_str=quatier_str.replace("marche","marché")
    if isinstance(quatier_str,str) and "new" in quatier_str and ("aeroport" in quatier_str or ("aéroport" in quatier_str or "airport" in quatier_str)):
        quatier_str="new town aéroport"
    if isinstance(quatier_str,str) and "new" in quatier_str and ("el" in quatier_str ):
        quatier_str="new bell"
    return quatier_str

In [ ]:
Base["Quartier de Résidence"]=Base["Quartier de Résidence"].apply(corriger_quartier)

In [ ]:
Base['Quartier de Résidence'].nunique()

In [ ]:
df =Base

# Étape 1 : Identifier les quartiers qui apparaissent dans plusieurs arrondissements
quartiers_repetes = df.groupby('Quartier de Résidence')['Arrondissement de résidence'].nunique() > 1
quartiers_repetes = quartiers_repetes[quartiers_repetes].index.tolist()

# Étape 2 : Pour chaque quartier répété, remplacer 'DOUALA' par l'autre arrondissement
for quartier in quartiers_repetes:
    # Trouver tous les arrondissements associés à ce quartier
    arrondissements = df[df['Quartier de Résidence'] == quartier]['Arrondissement de résidence'].unique()
    
    # Si 'DOUALA' est présent et qu'il y a un autre arrondissement
    if 'DOUALA' in arrondissements and len(arrondissements) > 1:
        autre_arrondissement = [arr for arr in arrondissements if arr != 'DOUALA'][0]
        df.loc[(df['Quartier de Résidence'] == quartier) & (df['Arrondissement de résidence'] == 'DOUALA'), 'Arrondissement de résidence'] = autre_arrondissement
observations_par_arrondissement = df['Arrondissement de résidence'].value_counts().reset_index()
observations_par_arrondissement.columns = ['Arrondissement de résidence', 'Nombre d\'observations']
observations_par_arrondissement

In [ ]:
dicoReligion= {
	"Baptist":"BAPTISTE",
	"CHRETIEN":"Chretien",
	"Chrétienne":"Chretien",
    "Chrétien":"Chretien",
	"Chrétien non précisé":"Chretien",
	"Chrétien pas précisé":"Chretien",
	"Loïc":"Laïc",
	"LOIQUE":"Laïc",
	"Non precise":"Pas Précisé",
	"R A S":"Pas Précisé",
	"PENTECOTISTE":"Pentecôtiste",
    "Pantecotiste":"Pentecôtiste",
	"Pentecotiste":"Pentecôtiste",
	"pentecôtiste":"Pentecôtiste",
	"Pentecôtistes":"Pentecôtiste",
	"aucune":"Non-croyant"
	}

dicoProfession= {
	"agent communal":"Agent communal",
	"agent de propreté":"Agent de propreté",
	"AGENT DE SECURITE":"Agent de securité",
	"Agent sécurité":"Agent de securité",
	"Agent de securite":"Agent de securité",
	"AGENT DE SÉCURITÉ":"Agent de securité",
	"Agent de sécurité hgoped":"Agent de securité",
	"Angent de securité":"Agent de securité",
	"ARGENT DE SÉCURITÉ ":"Agent de securité",
	"AGENT D'ENTRETIEN":"Agent d'entretien",
	"agent d'entretien":"Agent d'entretien",
	"AGENT TECHNIQUE":"Agent technique",
	"agent technique":"Agent technique",
	"ANGENT COMMUNALE":"Agent communal",
	"ARGENT COMMERCIAL":"Agent commercial",
	"assistant administratif":"Assistant administratif",
	"Assistant d'administratif":"Assistant administratif",
	"Assistante":"Assistant",
	"Assistante comptable":"Assistant comptable",
	"Assurent":"Assureur",
	"assureur":"Assureur",
	"ASSUREUR":"Assureur",
	"brasseur":"Brasseur",
	"BUSINESMAN":"Business Man",
	"BUSINESMAN":"Business man",
	"CABLEUR":"Cableur",
	"cadre commercial":"Cadre commercial",
	"CARRELEUR":"Carreleur",
	"CHAUDRONIER":"Chaudronnier",
	"CHAUDRONNIER":"Chaudronnier",
	"Chaudronnerie":"Chaudronnier",
	"CHAUFEUR":"Chauffeur",
	"CHAUFFEUR":"Chauffeur",
	"chauffeur":"Chauffeur",
	"CHEMINOT":"Cheminot",
	"chimiste":"Chimiste",
	"Chirurgien dentiste":"Chirurgien dentiste",
	"COIFFEUR":"Coiffeur",
	"COIFFEURS":"Coiffeur",
	"Coiffeuse":"Coiffeur",
	"COIFFEUSE":"Coiffeur",
	"coiffeuse":"Coiffeur",
	"COMMERCANT":"Commerçant (e)",
	"Vendeur":"Commerçant (e)",
	"VENDEUSE":"Commerçant (e)",
	"COMMERÇANTE":"Commerçant (e)",
	"Commerciale":"Commercial",
	"COMPTABLE":"Comptable",
	"comptable":"Comptable",
	"CONDUCTEUR":"Conducteur",
	"Conducteur engin":"Conducteur d'engin",
	"conducteur d'engin":"Conducteur d'engin",
	"Courtierd'assurance":"Courtier d'assurance",
	"COUTURIER":"Couturier",
	"COUTURIERE":"Couturier",
	"couturière":"Couturier",
	"CUISINIER":"Cuisinier",
	"Cultuvateur":"Cultivateur",
	"DEBROUILLARD":"Débrouillard",
	"Debrouillard":"Débrouillard",
	"DECLAMANT DOUANE":"Déclarant de douane",
	"DECLARANT DOUANE":"Déclarant de douane",
	"Declarant en douane":"Déclarant de douane",
	"Déclarant en douane":"Déclarant de douane",
	"Declarante en Douane":"Déclarant de douane",
	"DOCKER":"Docker",
	"Doker":"Docker",
	"Dolker":"Docker",
	"ELECTRICIEN":"Électricien",
	"Electricien":"Électricien",
	"Électricité":"Électricien",
	"Electronicien":"Électronicien",
	"Electrotechnicien":"Électrotechnicien",
	"Eleve":"Élève",
	"ELEVE":"Élève",
	"ElEVE":"Élève",
	"eleve":"Élève",
	"élève":"Élève",
	"Elève":"Élève",
	"EMPLOYE ENTREPRISE":"Employé (e)  dans une entreprise",
	"Enseignant":"Enseignant (e)",
	"Etudiant":"Etudiant (e)",
	"ETUDIANT":"Etudiant (e)",
	"FOOTBALLEUR":"Footballeur",
	"GESTIONNAIRE":"Gestionnaire",
	"gestionnaire":"Gestionnaire",
	"Gestionnaire ressources humaines":"Gestionnaire de ressources humaines",
	"HOMME D'AFFAIRE":"Homme d'affaire",
	"Hotelier":"Hôtelier",
	"HOTELIER":"Hôtelier",
	"HOTELLIER":"Hôtelier",
	"Hoteliere":"Hôtelier",
	"Hoteluere":"Hôtelier",
	"hoteliere":"Hôtelier",
	"Hôtelière":"Hôtelier",
	"IMPRIMEUR":"Imprimeur",
	"Infirmièr":"Infirmier",
	"INFIRMIER":"Infirmier",
	"Infor'aticien":"Informaticien",
	"INFORMATICIEN":"Informaticien",
	"Informatien":"Informaticien",
	"Informatticien":"Informaticien",
	"INFORMATICIEN DE RESEAU":"Informaticien de réseau",
	"Ingenieur":"Ingénieur ",
	"Ingénieur civil":"Ingénieur en génie civil",
	"Ingénieur génie civil":"Ingénieur en génie civil",
	"Ingénieur réseaux et telecom":"Ingénieur réseaux et telecoms",
	"MACHINISTE":"Machiniste",
	"Logiticien":"Logisticien",
	"MAGASINIER":"Magasinier",
	"Maintenancier ind":"Maintenancier industriel",
	"manoeuvre":"MANŒUVRE",
	"MARIN":"Marin",
	"marrin":"Marin",
	"Mecanicien":"Mécanicien",
	"MECANICIEN":"Mécanicien",
	"MÉCANICIEN":"Mécanicien",
	"Medecin":"Médecin",
	"médecin":"Médecin",
	"MENAGERE":"Ménagère",
	"Ménuisier":"Menuisier",
	"MENUISIER":"Menuisier",
	"Menusier":"Menuisier",
	"Mnuisier":"Menuisier",
	"MENUSIER":"Menuisier",
	"Missionary":"Missionnaire",
	"missionnaire":"Missionnaire",
	"MOTO TAXI":"Moto taximan",
	"MOTO TAXIMAN":"Moto taximan",
	"Motoman":"Moto taximan",
	"Mototaximan":"Moto taximan",
	"Moto-taximan":"Moto taximan",
	"Bensikineur":"Moto taximan",
	"NON PRECISE":"Non precisé",
	"Pas mentionné":"Non precisé",
	"Pas précise":"Non precisé",
	"Pas précisée":"Non precisé",
	"PASTEUR":"Pasteur",
	"Patissier":"Pâtissier",
	"PEINTRE":"Peintre",
	"photographe":"Photographe",
	"PLOMBIER":"Plombier",
	"R A S":"Non precisé",
	"RAS":"Non precisé",
	"Ras":"Non precisé",
	"Realisateur":"Réalisateur",
	"Chomeur":"Sans emploi",
	"Chômeur":"Sans emploi",
	"Sans Emplois":"Sans emploi",
	"SCULPTEURS":"Sculpteur",
	"Secretaire":"Secrétaire",
	"Sérigraphie":"Sérigraphe",
	"SOUDEUR":"Soudeur",
	"TAXIMAN":"Taximan",
	"taximan":"Taximan",
	"TECHNICIEN":"Technicien",
	"Technicien en Batiment":"Technicien en batiment",
	"TECHNICIEN EN BATIMENT":"Technicien en batiment",
	"Technicien en genie civil":"Technicien génie civil",
	"TECHNICIEN GENIE CIVIL":"Technicien génie civil",
	"Tolier":"Tôlier",
	"Vitrié":"Vitrier",
	"VITRIER":"Vitrier"

	}

def corriger_hemoglobine(taux_str):
    if isinstance(taux_str,str) and "g/dl" in taux_str:
        taux_str=taux_str.replace('g/dl','')
        taux_str=taux_str.replace(',','.')
    return taux_str

In [ ]:
# Fonction pour corriger les années (ex: '93' devient '1993')
def corriger_annee(date_str):
    if isinstance(date_str, str):  # Si c'est du texte
        parts = date_str.split('-')  # Séparer jour/mois/année
        if len(parts) == 3:  # Si format correct (3 parties)
            year = parts[0]  # On prend l'année
            # Si année sur 2 chiffres (ex: '93')
            if len(year) == 2:
                return f"19{year}-{parts[1]}-{parts[2]}"  # Devient '1993'
            # Si année mal écrite (ex: '093')
            elif len(year) < 4:
                return f"19{year.zfill(2)}-{parts[1]}-{parts[2]}"  # Devient '1993'
    if isinstance(date_str,str) and "0019" in date_str:
        date_str=date_str.replace('0019','2019')
    return date_str  # Si pas de problème, on garde tel quel

In [ ]:
Base["Si autres raison préciser"].value_counts()

### CREATION DE GRAPHIQUES AVEC PLOTLY.EXPRESS

In [ ]:
fig_anneau1 = px.pie(Base, names = "ÉLIGIBILITÉ AU DON.", hole = 0.4,color_discrete_sequence=["#0ABAB5","#FF6F61","#F9E79F"],title = "Répartition des donneurs éligibles et non éligibles")
fig_anneau1.add_annotation(
    text="Le nombre d’éligibles est visiblement supérieur au nombre de temporairement et définitivement non éligibles.",
    x=1.6, y=-0.1,  # Position du texte
    showarrow=False,
    font=dict(size=8, color="black"),
    align="center"
)
fig_anneau1.update_traces(pull=[0.1,0.1])
fig_anneau1.show()

In [ ]:
Base['Si autres raison préciser'].unique()

In [ ]:
Base['Si autres raison préciser']=Base['Si autres raison préciser'].replace("EU UNE ENDOSCOPIE ( FIBROSCOPIE,  GASTROSCOPIE, COLOSCOPIE .......)","EU UNE ENDOSCOPIE")
Base['Si autres raison préciser']=Base['Si autres raison préciser'].replace('Consommation de drogue ',"DROGUES")
Base['Si autres raison préciser']=Base['Si autres raison préciser'].replace('ETE TRAITE PAR ACUPUNCTURE','TRAITE PAR ACUPUNCTURE')
Base['Si autres raison préciser']=Base['Si autres raison préciser'].replace('PAS D INFORMATION SUR SON DOSSIER','Aucune information ')

In [ ]:
Base['Si autres raison préciser'].unique()

In [ ]:
Conditions_sante = Base[Base["ÉLIGIBILITÉ AU DON."]=="Définitivement non-eligible"]
Conditions_sante.drop(["Date de remplissage de la fiche", "Date de naissance","Niveau d'etude", "Genre", "Taille", "Poids",
       "Situation Matrimoniale (SM)", "Profession",
       "Arrondissement de résidence", "Quartier de Résidence", "Nationalité",
       "Religion", "A-t-il (elle) déjà donné le sang",
       "Si oui preciser la date du dernier don.", "Taux d’hémoglobine",
       "Raison indisponibilité  [Est sous anti-biothérapie  ]",
       "Raison indisponibilité  [Taux d’hémoglobine bas ]",
       "Raison indisponibilité  [date de dernier Don < 3 mois ]",
       "Raison indisponibilité  [IST récente (Exclu VIH, Hbs, Hcv)]",
       "Date de dernières règles (DDR) ",
       "Raison de l’indisponibilité de la femme [La DDR est mauvais si <14 jour avant le don]",
       "Raison de l’indisponibilité de la femme [Allaitement ]",
       "Raison de l’indisponibilité de la femme [A accoucher ces 6 derniers mois  ]",
       "Raison de l’indisponibilité de la femme [Interruption de grossesse  ces 06 derniers mois]",
       "Raison de l’indisponibilité de la femme [est enceinte ]",
       "Autre raisons,  preciser", 'Sélectionner "ok" pour envoyer'],axis=1,inplace = True)
Conditions_sante

In [ ]:
Conditions_sante["Si autres raison préciser"]=Conditions_sante["Si autres raison préciser"].replace('Aucune information ','Aucune')
Conditions_sante["Si autres raison préciser"]=Conditions_sante["Si autres raison préciser"].replace('PAS D INFORMATION SUR SON DOSSIER','Aucune')

In [ ]:
liste_raisons = ["Antécédent de transfusion", "Porteur(HIV,hbs,hcv)","Opéré","Drepanocytaire","Diabétique","Hypertendus","Asthmatiques","Cardiaque","Tatoué","Scarifié"]
dico_de_raisons = dict(zip(Conditions_sante.columns[1:11],liste_raisons))
dico_de_raisons

In [ ]:
def combiner_raisons(row):
    raisons = [dico_de_raisons[col] for col in Conditions_sante.columns[1:11] if row[col] == 'Oui']
    if pd.notna(row["Si autres raison préciser"]):
        raisons.append(row["Si autres raison préciser"])
    return '-'.join(map(str,raisons)) if raisons else 'Aucune'
Conditions_sante["Raisons de non-éligibilité totale"]=Conditions_sante.apply(combiner_raisons, axis = 1)

In [ ]:
Conditions_sante["Raisons de non-éligibilité totale"].value_counts()

In [ ]:
fig_sunburst1 = px.sunburst(Conditions_sante,path=["Raisons de non-éligibilité totale"],title="Répartition des raisons de non éligibilité totale")
fig_sunburst1.add_annotation(
    text="Les antécédents de transfusion, les problèmes cardiaques et les opérations subies sont les raisons d'indisponibilité les plus fréquentes",
    x=0.5, y=-0.2,  # Position du texte
    showarrow=False,
    font=dict(size=8, color="black"),
    align="center"
)
fig_sunburst1.show()

In [ ]:
Base

In [ ]:
Base["Date de remplissage de la fiche"]=Base["Date de remplissage de la fiche"].apply(corriger_annee)

In [ ]:
Base["Date de remplissage de la fiche"]

In [ ]:
Base["Date de remplissage de la fiche"]=pd.to_datetime(Base["Date de remplissage de la fiche"],errors='coerce')
Base["Date de naissance"]=pd.to_datetime(Base["Date de naissance"],errors='coerce')

In [ ]:
Base

In [ ]:
Base["Âge"]= Base.apply(
    lambda row: (row['Date de remplissage de la fiche'].year - row['Date de naissance'].year) - 
               ((row['Date de remplissage de la fiche'].month, row['Date de remplissage de la fiche'].day) < (row['Date de naissance'].month, row['Date de naissance'].day))
    if pd.notna(row['Date de naissance']) and pd.notna(row['Date de remplissage de la fiche']) else None,
    axis=1
)


In [ ]:
Base[Base["Âge"]<=0]

In [ ]:
# Nettoyage et préparation des variables
Base["Date de naissance"] = pd.to_datetime(Base["Date de naissance"], errors="coerce")
Base["Profession"]=Base["Profession"].replace(dicoProfession)
Base["Religion"]=Base["Religion"].replace(dicoReligion)
Base['Taux d’hémoglobine']=Base['Taux d’hémoglobine'].apply(corriger_hemoglobine)
#Base['Taux d’hémoglobine']=Base['Taux d’hémoglobine'].replace(',','.')
#Base['Taux d’hémoglobine']=Base['Taux d’hémoglobine'].replace('14 .6','14.6')
Base['Taux d’hémoglobine']=pd.to_numeric(Base['Taux d’hémoglobine'],errors = 'coerce')
Base['Taux d’hémoglobine']=Base['Taux d’hémoglobine'].fillna(Base['Taux d’hémoglobine'].median())
#Base['Taux d’hémoglobine']=Base['Taux d’hémoglobine'].astype(float)

In [ ]:
Base2=Base
Base2=Base2.rename(columns={'Taux d’hémoglobine':"Taux_d_hémoglobine","Situation Matrimoniale (SM)":"Situation_Matrimoniale",
                      "A-t-il (elle) déjà donné le sang":"Déjà_donné_le_sang","Niveau d'etude":"Niveau_d_etude"})
# Sélection des variables pertinentes
features = ["Âge",  "Niveau_d_etude","Genre", "Situation_Matrimoniale","Déjà_donné_le_sang",
                'Religion',"Taux_d_hémoglobine",'Profession']
target ='ÉLIGIBILITÉ AU DON.'# Modifier selon le nom réel de la colonne cible

Base2 = Base2[features + [target]]
# Calcul de la médiane
hb_median = Base2['Taux_d_hémoglobine'].median()

# Encodage One-Hot des variables catégoriques
cat_cols = ["Niveau_d_etude", "Genre", "Situation_Matrimoniale","Déjà_donné_le_sang",'Religion','Profession']
Base2[cat_cols]=Base2[cat_cols].fillna(Base2[cat_cols].mode().iloc[0])
encoder = OneHotEncoder(sparse=False, drop="first",handle_unknown="ignore")
encoded_data = encoder.fit_transform(Base2[cat_cols])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(cat_cols))

# imputation de la moyenne aux donnees manquantesdes variables numériques
Base2["Âge"] =Base2["Âge"].fillna(Base2["Âge"].median() )

# Remplacement des colonnes encodées
Base2 = Base2.drop(columns=cat_cols)
Base2 = pd.concat([Base2.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Séparation des données en train/test
X = Base2.drop(columns=[target])
y = Base2[target].map({"Eligible": 1, "Définitivement non-eligible": 0,"Temporairement Non-eligible":2})  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calcul des poids des classes
classes = np.unique(y_train)
weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

# Modèle Random Forest optimisé
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=7,
    min_samples_split=10,
    class_weight=class_weights,
    random_state=42
)

# Entraînement du modèle
model.fit(X_train, y_train)

# Évaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nMatrice de confusion:")
print(confusion_matrix(y_test, y_pred))


# Sauvegarde du modèle et de l'encodeur
joblib.dump(model, "model.pkl")
joblib.dump(encoder, "encoder.pkl")

In [ ]:
liste_raisons
Nbre_Oui = [(Conditions_sante[col]=="Oui").sum() for col in Conditions_sante.columns[1:11]]
dico_r = dict(zip(liste_raisons,Nbre_Oui))
Nbre_Autres_raisons = [(Conditions_sante["Si autres raison préciser"]==x).sum() for x in Conditions_sante["Si autres raison préciser"].unique()]
dico_a = dict(zip(Conditions_sante["Si autres raison préciser"].unique(),Nbre_Autres_raisons))
del dico_a[next(iter(dico_a))]
dico_a

In [ ]:
Occur_Raison_dico = {**dico_r,**dico_a}
Occur_Raison_dico

In [ ]:
Occur_Raison_df = pd.DataFrame(list(Occur_Raison_dico.items()),columns=["Raisons d'indisponibilité totale","Occurrences"])

In [ ]:
fig_barh1 = px.bar(Occur_Raison_df, y="Raisons d'indisponibilité totale",x="Occurrences",orientation='h',opacity=0.5,title="Occurrences des raisons d'indisponibilité totale")
fig_barh1.update_traces(marker_line_color='blue',marker_line_width=2)
fig_barh1.show()

In [ ]:
fig_box1 = px.box(Base,x="ÉLIGIBILITÉ AU DON.",y="Taux d’hémoglobine",color="ÉLIGIBILITÉ AU DON.",color_discrete_sequence=["#0ABAB5","red","purple"],title="Distribution-Eligibilité au don/Taux d'hémoglobine",height=600)
fig_box1.update_yaxes(range=[5,20.5])
fig_box1.show()

In [ ]:
Base.columns

In [ ]:
fig_violin1 = px.violin(Base[Base["Âge"]>0],x="ÉLIGIBILITÉ AU DON.",y="Âge",color="ÉLIGIBILITÉ AU DON.",color_discrete_sequence=["#0ABAB5","#FF6F61","#F9E79F"],title ="Distribution de l'éligibilité au don de sang en fonction de l'âge")
fig_violin1.show()

In [ ]:
Base_don_deja=Base[Base["A-t-il (elle) déjà donné le sang"]=="Oui"]
Base_don_deja["A-t-il (elle) déjà donné le sang"]

In [ ]:
Base_don_deja["Si oui preciser la date du dernier don."]

In [ ]:
Base_don_deja=Base_don_deja[["Genre","Arrondissement de résidence","Quartier de Résidence","A-t-il (elle) déjà donné le sang","Si oui preciser la date du dernier don.","ÉLIGIBILITÉ AU DON.","Âge"]]

In [ ]:
Base_don_deja["Si oui preciser la date du dernier don."]=pd.to_datetime(Base_don_deja["Si oui preciser la date du dernier don."],errors='coerce')
Base_don_deja["Si oui preciser la date du dernier don."]

In [ ]:
Base_don_deja[Base_don_deja["Arrondissement de résidence"]=="DOUALA"]["Quartier de Résidence"].unique()

In [ ]:
Annees=Base_don_deja["Si oui preciser la date du dernier don."].dt.year.value_counts().sort_index()
Annees_dons_occur=pd.DataFrame()
Annees_dons_occur["Années"]=Annees.index
Annees_dons_occur["Années"]=Annees_dons_occur["Années"].astype(int)
Annees_dons_occur["Nombre de dons"]=Annees.values
Annees_dons_occur

In [ ]:
fig_scatter1=px.scatter(Annees_dons_occur,x="Années",y="Nombre de dons",color_discrete_sequence=["red"],labels={"x":"Annee","y":"Nombre de dons de sang enregistrés"},title="Evolution du nombre de dons de sang enregistrés")
fig_scatter1.update_xaxes(range=[2004.5,2020.5],dtick=1)
fig_scatter1.update_traces(mode='lines+markers')
fig_scatter1.show()

In [ ]:
tableau_croise = pd.crosstab(Base_don_deja["Arrondissement de résidence"],Base_don_deja["A-t-il (elle) déjà donné le sang"])
tableau_croise

In [ ]:
fig_heatmap = px.imshow(tableau_croise,text_auto = True, color_continuous_scale="RdBu",labels=dict(x="A déja donné du sang?",y="Arrondissement de résidence",
color="Frequence"), title="Relation entre arrondissement de résidence et don de sang")
fig_heatmap.show()

In [ ]:
fig_sunburst2 = px.sunburst(Base,path=["ÉLIGIBILITÉ AU DON.","A-t-il (elle) déjà donné le sang"],title="Eligibilité-Etat de participation aux précédentes campagnes")
fig_sunburst2.show()

In [ ]:
mois_dons_occur=Base_don_deja["Si oui preciser la date du dernier don."].dt.month_name().value_counts()

In [ ]:
fig_barv1=px.bar(x=mois_dons_occur.index,y=mois_dons_occur.values,labels={"x":"Mois","y":"Nombre de dons de sang enregistrés"},title="Analyse des périodes annuelles propices aux dons")
fig_barv1.show()          

### API ET PREDICTION

In [ ]:
# Charger le modèle et l'encodeur
model = joblib.load("model.pkl")  #  modèle est bien sauvegardé
encoder = joblib.load("encoder.pkl")  # encodeur est bien sauvegardé

# Initialiser FastAPI
app1 = FastAPI()

# Définir le format des entrées attendues
class InputData(BaseModel):
    Âge: int
    Niveau_d_etude: str
    Genre: str
    
    Situation_Matrimoniale: str
    Déjà_donné_le_sang: str
    Religion: str
    Taux_d_hémoglobine: float
    Profession: str

@app1.post("/predict")
def predict(data: InputData):

    try:
        df = pd.DataFrame([data.dict()])
        df["Profession"]=df["Profession"].replace(dicoProfession)
        df["Religion"]=df["Religion"].replace(dicoReligion)
        
        # Vérifier les variables catégorielles et gérer les valeurs inconnues
        cat_cols = [ "Niveau_d_etude", "Genre","Situation_Matrimoniale", "Déjà_donné_le_sang", "Religion", "Profession"]
        #for col in cat_cols:
            #if df[col][0] not in encoder.categories_[cat_cols.index(col)]:
                #df[col] = "Autre"  # Catégorie par défaut

        # Encodage One-Hot
        encoded_data = encoder.transform(df[cat_cols])
        encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(cat_cols))

        # Concaténer les variables numériques et catégorielles
        df = df.drop(columns=cat_cols)
        df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
        
        # Prédiction
        # prediction = model.predict(df)[0]
        prediction = model.predict(df)
        if prediction==1:
            return "Éligible"
        if prediction==0:
            return "Définitivement non-eligible"
        if prediction==2:
            return "Temporairement Non-eligible"
    
    except Exception as e:
        return {"error": str(e)}
def run_api():
    uvicorn.run(app1, host="0.0.0.0", port=8001)

thread = threading.Thread(target=run_api)
thread.start()

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut
import time

# Charger les données
df = Base
# Nettoyer les adresses
df['Adresse'] = df['Arrondissement de résidence'] + ', ' + df['Quartier de Résidence']
df['Adresse'] = df['Adresse'].str.strip()  # Supprimer les espaces inutiles
df['Adresse'] = df['Adresse'].str.replace(r'\s+', ' ', regex=True)  # Remplacer les espaces multiples par un seul espace

# Initialiser le géocodeur avec un délai d'attente plus long
geolocator = Nominatim(user_agent="donneurs_sang", timeout=30)  # 10 secondes
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Charger ou créer un cache
try:
    df_cached = pd.read_csv('geocoded_cache_LEBON9.csv')
    print("Cache chargé. Utilisation des données existantes.")
except FileNotFoundError:
    df_cached = pd.DataFrame(columns=['Adresse', 'latitude', 'longitude'])
    print("Aucun cache trouvé. Début du géocodage.")

# Fonction pour géocoder une adresse avec gestion des erreurs
n=0
def geocode_address(address):
    global n
    try:
        # Vérifier si l'adresse est déjà dans le cache
        if address in df_cached['Adresse'].values:
            print(f"Adresse trouvée dans le cache : {address}")
            return df_cached.loc[df_cached['Adresse'] == address, ['latitude', 'longitude']].values[0]
        else:
            print(f"Géocodage de l'adresse : {address}")
            location = geocode(address)
            if location:
                # Ajouter les résultats au cache
                df_cached.loc[len(df_cached)] = [address, location.latitude, location.longitude]
                return location.latitude, location.longitude
            else:
                n=n+1
                print(f"Géocodage échoué pour : {address}", n)
                return None, None
    except (GeocoderUnavailable, GeocoderTimedOut) as e:
        print(f"Erreur de géocodage pour {address} : {e}")
        return None, None
    except Exception as e:
        print(f"Erreur inattendue pour {address} : {e}")
        return None, None

# Appliquer la fonction de géocodage à toutes les adresses
df['latitude'], df['longitude'] = zip(*df['Adresse'].apply(geocode_address))

# Sauvegarder le cache
df_cached.to_csv('geocoded_cache_LEBON9.csv', index=False)
print("Cache sauvegardé.")

# Afficher les résultats
print(df[['Adresse', 'latitude', 'longitude']])

# Sauvegarder les résultats dans un fichier Excel avec toutes les colonnes
df.to_excel('LE_FICHIER _AVEC_LES_MEILLEURS_GEOCODES.xlsx', index=False)
print("Résultats du géocodage enregistrés dans 'LE_BON_FICHIER.xlsx'.")

### CARTOGRAPHIE

In [ ]:
Base_Long_Lat=pd.read_excel("LE_FICHIER _AVEC_LES_MEILLEURS_GEOCODES.xlsx")
Base_Long_Lat.index=Base_Long_Lat.index+1

In [ ]:
Base_Long_Lat["Quartier de Résidence"]=Base_Long_Lat["Quartier de Résidence"].apply(corriger_quartier)

In [ ]:
# Étape 1 : Identifier les quartiers qui apparaissent dans plusieurs arrondissements
quartiers_repetes = Base_Long_Lat.groupby('Quartier de Résidence')['Arrondissement de résidence'].nunique() > 1
quartiers_repetes = quartiers_repetes[quartiers_repetes].index.tolist()

# Étape 2 : Pour chaque quartier répété, remplacer 'DOUALA' par l'autre arrondissement
for quartier in quartiers_repetes:
    # Trouver tous les arrondissements associés à ce quartier
    arrondissements = Base_Long_Lat[Base_Long_Lat['Quartier de Résidence'] == quartier]['Arrondissement de résidence'].unique()
    
    # Si 'DOUALA' est présent et qu'il y a un autre arrondissement
    if 'DOUALA' in arrondissements and len(arrondissements) > 1:
        autre_arrondissement = [arr for arr in arrondissements if arr != 'DOUALA'][0]
        Base_Long_Lat.loc[(Base_Long_Lat['Quartier de Résidence'] == quartier) & (Base_Long_Lat['Arrondissement de résidence'] == 'DOUALA'), 'Arrondissement de résidence'] = autre_arrondissement
observations_par_arrondissement = Base_Long_Lat['Arrondissement de résidence'].value_counts().reset_index()
observations_par_arrondissement.columns = ['Arrondissement de résidence', 'Nombre d\'INDIVIDUS']
observations_par_arrondissement

In [ ]:
Base_Long_Lat['latitude']=Base_Long_Lat.groupby("Arrondissement de résidence")['latitude'].transform(lambda x: x.fillna(x.mean()))
Base_Long_Lat['longitude']=Base_Long_Lat.groupby("Arrondissement de résidence")['longitude'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
Base_Long_Lat[Base_Long_Lat['latitude'].isna()]

In [ ]:
Base_Long_Lat=Base_Long_Lat.dropna(subset=['latitude'])

In [ ]:
# 1. Correction pour la date de naissance
# Convertir en texte, appliquer la correction
Base_Long_Lat['Date de naissance'] = Base_Long_Lat['Date de naissance'].astype(str).apply(corriger_annee)
# Transformer en vraie date
Base_Long_Lat['Date de naissance'] = pd.to_datetime(Base_Long_Lat['Date de naissance'], errors='coerce')
# Enlever l'heure, garder juste la date
Base_Long_Lat['Date de naissance'] = Base_Long_Lat['Date de naissance'].dt.strftime('%Y-%m-%d')

# 2. Même chose pour la date de remplissage
Base_Long_Lat['Date de remplissage de la fiche'] = (
    Base_Long_Lat['Date de remplissage de la fiche']
    .astype(str)
    .apply(corriger_annee)
)
Base_Long_Lat['Date de remplissage de la fiche'] = pd.to_datetime(
    Base_Long_Lat['Date de remplissage de la fiche'], 
    errors='coerce'
)
Base_Long_Lat['Date de remplissage de la fiche'] = (
    Base_Long_Lat['Date de remplissage de la fiche']
    .dt.strftime('%Y-%m-%d')
)

In [ ]:
# Créer l'application Dash principale
app = dash.Dash(__name__)

# Layout principal pour l'application avec la barre de navigation
app.layout = html.Div([
    dcc.Location(id='url',refresh=False),
dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("> Tableau de bord", href="/page-1", style={
            "fontWeight": "800",
            "fontSize": "2rem",
            "margin": "0 15px",
            "padding": "10px 15px",
            "transition": "all 0.3s ease",
            "color": "white"
        })),
        dbc.NavItem(dbc.NavLink("> Cartographie de la répartition des donneurs", href="/page-2", style={
            "fontWeight": "800",
            "fontSize": "2rem", 
            "margin": "0 15px",
            "padding": "10px 15px",
            "color": "white"
        })),
        dbc.NavItem(dbc.NavLink("> Prédiction de l'éligibilité", href="/page-3", style={
            "fontWeight": "800",
            "fontSize": "2rem",
            "margin": "0 15px",
            "padding": "10px 15px",
            "color": "white"
        }))
    ],
    brand="DASHBOARD MULTIPAGES: Veuillez choisir une page à consulter",
    brand_href="/",
    brand_style={
        "fontSize": "3rem",
        "fontWeight": "800",
        "color": "white", 
        "padding": "10px",  
        "textAlign": "center"
    },
    color="primary",
    dark=True,
    sticky="top",
    style={
        "backgroundColor": "#cc0000",
        "padding": "0.8rem 2rem",
        "boxShadow": "0 2px 10px rgba(0,0,0,0.1)",
        "fontFamily": "'Segoe UI', Arial, sans-serif"  
    }
),
html.Div(id='page-content')
])

# Fonction pour les pages
@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname')
)
def display_page(pathname):
    if pathname == '/page-1':
        layout = html.Div(
            children=[
                html.H1("TABLEAU DE BORD : CAMPAGNE DE DON DE SANG", style={"textAlign": "center"}),
                
                # Première ligne : Graphique en anneau + Sunburst
                html.Div([
                    html.Div(dcc.Graph(figure=fig_anneau1), style={"width": "48%", "display": "inline-block", "border": "1px solid lightgray", "padding": "10px"}),
                    html.Div(dcc.Graph(figure=fig_sunburst1), style={"width": "48%", "display": "inline-block", "border": "1px solid lightgray", "padding": "10px"})
                ], style={"display": "flex", "justifyContent": "space-between"}),
        
            # Deuxième ligne : Bar Chart sur toute la largeur
                html.Div([
                    dcc.Dropdown(
                        id="raisons-indisponibilité",
                        options=[{'label': modalite, 'value': modalite} for modalite in Occur_Raison_df["Raisons d'indisponibilité totale"].unique()],
                        value=Occur_Raison_df["Raisons d'indisponibilité totale"].unique().tolist(),
                        multi=True,
                        clearable=False,
                        style={'width': '100%'}
                    ),
                    dcc.Graph(id="barh1-graph", figure=fig_barh1)
                ], style={"width": "100%", "border": "1px solid lightgray", "padding": "10px", "marginTop": "10px"}),
        
            # Troisième ligne : Dropdown + Boxplot
                html.Div([
                    html.Div([
                        dcc.Dropdown(
                            id="Eligibilite-Hémoglobine",
                            options=[{'label': cat, 'value': cat} for cat in Base["ÉLIGIBILITÉ AU DON."].unique()],
                            value=['Eligible', 'Temporairement Non-eligible', 'Définitivement non-eligible'],
                            multi=True,
                            clearable=False,
                            style={'width': '100%'}
                        ),
                        dcc.Graph(id="box1-graph", figure=fig_box1)
                    ], style={"width": "48%", "display": "inline-block", "border": "1px solid lightgray", "padding": "10px"}),
                    
                    html.Div([
                        dcc.Dropdown(
                            id="Eligibilite-Don-avant",
                            options=[{'label': 'Éligibilité au don > Donateur ultérieur', 'value': 'CS'},
                                     {'label': 'Donateur ultérieur > Éligibilité au don', 'value': 'SC'}],
                            value='CS',
                            clearable=False,
                            style={'width': '100%'}
                        ),
                        dcc.Graph(id="sunburst2-graph", figure=fig_sunburst2)
                    ], style={"width": "48%", "display": "inline-block", "border": "1px solid lightgray", "padding": "10px"})
                ], style={"display": "flex", "justifyContent": "space-between", "marginTop": "10px"}),
        
            # Quatrième ligne : Checklist + Heatmap + Bar Chart
                html.Div([
                    html.Div([
                        dcc.Checklist(
                            id='arrondissements-checklist',
                            options=[{'label': k, 'value': k} for k in tableau_croise.index],
                            value=list(tableau_croise.index),
                            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                        ),
                        dcc.Graph(id="heatmap-graph", figure=fig_heatmap)
                    ], style={"width": "48%", "display": "inline-block", "border": "1px solid lightgray", "padding": "10px"}),
                    
                    html.Div(dcc.Graph(figure=fig_barv1), style={"width": "48%", "display": "inline-block", "border": "1px solid lightgray", "padding": "10px"})
                ], style={"display": "flex", "justifyContent": "space-between", "marginTop": "10px"}),
        
            # Cinquième ligne : RangeSlider + Scatter Plot
                html.Div([
                    dcc.Graph(id='scatter-graph', figure=fig_scatter1),
                    dcc.RangeSlider(
                        id='year-range-slider',
                        min=1999,
                        max=int(Annees_dons_occur["Années"].max()),
                        step=1,
                        marks={int(year): str(year) for year in Annees_dons_occur["Années"]},
                        value=[int(Annees_dons_occur["Années"].min()), int(Annees_dons_occur["Années"].max())],
                        tooltip={"placement": "bottom", "always_visible": True}
                    )
                ], style={"width": "100%", "border": "1px solid lightgray", "padding": "10px", "marginTop": "10px"})
            ]
        )
        return layout
    elif pathname == '/page-2':
        os.makedirs('templates', exist_ok=True)  # Crée un dossier pour les cartes
    #app = dash.Dash(__name__)  # Crée l'application
    #app.title = "Dashboard Donneurs de Sang"  # Titre de la page
    
    # On utilise directement Base_Long_Lat comme source de données
        arrondissements = sorted(Base_Long_Lat['Arrondissement de résidence'].str.upper().unique())
    #C'est Liste des arrondissements ordonnés dans l'ordre alphabétique
    
    # Structure de la page web
        layout = html.Div([
            html.H1("Répartition Géographique des Donneurs de Sang"),  # Grand titre
        
        # Menu pour choisir l'arrondissement
            html.Label("Sélectionnez un arrondissement :"),
            dcc.Dropdown(
                id='arrondissement-dropdown',
                options=[{'label': arr, 'value': arr} for arr in arrondissements],
                value=arrondissements[0],  # Premier arrondissement sélectionné par défaut
                clearable=False
            ),
        
        # Menu pour choisir le quartier (sera mis à jour après)
            html.Label("Sélectionnez un quartier :"),
            dcc.Dropdown(id='quartier-dropdown', clearable=False),
        
        # Emplacement pour la carte
            html.Iframe(id='folium-map', width='100%', height='600px')
        ])
        return layout
    elif pathname == '/page-3':
        #response = requests.get("http://127.0.0.1:8001/predict", json=data)
        layout = html.Div([
            html.H1("Prédiction d'Éligibilité au Don de Sang", style={'textAlign': 'center', 'marginBottom': 30}),
    
        html.Div([
        # Première ligne (Age, Taux HB, Genre)
            html.Div([
                html.Div([
                    html.Label(["Âge", html.Span("*", style={'color': 'red'})], 
                             style={'fontWeight': 'bold', 'marginBottom': 5}),
                             dcc.Input(id="age", type="number", placeholder="Entrez votre âge",
                             style={'width': '90%', 'padding': '8px'})
                ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            
                html.Div([
                    html.Label("Taux d'hémoglobine", style={'fontWeight': 'bold', 'marginBottom': 5}),
                                 dcc.Input(id="taux_hb", type="number", placeholder="Entrez votre taux",
                                 style={'width': '90%', 'padding': '8px'})
                    ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            
                html.Div([
                    html.Label(["Genre", html.Span("*", style={'color': 'red'})], 
                                 style={'fontWeight': 'bold', 'marginBottom': 5}),
                                 dcc.Dropdown(id="genre", options=[{"label": g, "value": g} for g in ["Homme", "Femme"]], 
                                 placeholder="Sélectionnez",
                                 style={'width': '90%'})
                    ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                ], style={'marginBottom': 25, 'display': 'flex', 'justifyContent': 'space-between'}),
        
        # Deuxième ligne (Niveau étude, Situation matrimoniale, Don sang)
            html.Div([
                html.Div([
                    html.Label(["Niveau d'étude", html.Span("*", style={'color': 'red'})], 
                                 style={'fontWeight': 'bold', 'marginBottom': 5}),
                                 dcc.Dropdown(id="niveau_etude", 
                                 options=[{"label": n, "value": n} for n in ["Aucun","Pas Précisé","Primaire","Secondaire", "Universitaire"]], 
                                 placeholder="Sélectionnez",
                                 style={'width': '100%'})
                ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '3%'}),
            
                html.Div([
                    html.Label(["Situation matrimoniale", html.Span("*", style={'color': 'red'})], 
                                 style={'fontWeight': 'bold', 'marginBottom': 5}),
                                 dcc.Dropdown(id="situation_matrimoniale", 
                                 options=[{"label": s, "value": s} for s in ["Célibataire", "Marié(e)","Divorcé(e)","veuf (veuve)"]], 
                                 placeholder="Sélectionnez",
                                 style={'width': '100%'})
                ], style={'width': '30%', 'display': 'inline-block', 'marginRight': '3%'}),
            
                html.Div([
                    html.Label(["A déjà donné le sang", html.Span("*", style={'color': 'red'})], 
                                 style={'fontWeight': 'bold', 'marginBottom': 5}),
                                 dcc.Dropdown(id="don_sang", options=[{"label": d, "value": d} for d in ["Oui", "Non"]], 
                                 placeholder="Sélectionnez",
                                 style={'width': '100%'})
                ], style={'width': '30%', 'display': 'inline-block'}),
            ], style={'marginBottom': 20}),
        
        # Troisième ligne (Religion et Profession)
            html.Div([
                html.Div([
                    html.Label(["Religion", html.Span("*", style={'color': 'red'})], 
                             style={'fontWeight': 'bold', 'marginBottom': 5}),
                             dcc.Dropdown(
                                id="religion", 
                                options=[{"label": r, "value": r} for r in [
                                "Non-croyant","Pas Précisé","Chrétien (Catholique)", "Musulman",
                                "Chretien (Ne de nouveau)","Chretien (Protestant )","Chrétien",
                                "Chretien (témoin de Jéhovah)", "Autre"
                                ]], 
                                placeholder="Sélectionnez",
                                style={'width': '100%'}
                            ),
                            dcc.Input(id="religion_autre", type="text", placeholder="Précisez votre religion si 'Autre'", 
                                style={'width': '100%', 'padding': '8px', 'marginTop': 10, 'display': 'none'})
                ], style={'width': '48%', 'display': 'inline-block', 'marginRight': '4%'}),
            
                html.Div([
                    html.Label(["Profession", html.Span("*", style={'color': 'red'})], 
                             style={'fontWeight': 'bold', 'marginBottom': 5}),
                            dcc.Dropdown(
                                id="profession", 
                                options=[{"label": p, "value": p} for p in [
                                "Agent de sécurité", "Médecin","Élève","Sans Emploi",
                                "Etudiant (e)","Enseignant (e)","Employé (e) dans une entreprise","Commerçant (e)","Autre"
                                ]], 
                                placeholder="Sélectionnez",
                                style={'width': '100%'}
                            ),
                            dcc.Input(id="profession_autre", type="text", placeholder="Précisez votre profession si 'Autre'", 
                            style={'width': '100%', 'padding': '8px', 'marginTop': 10, 'display': 'none'})
                ], style={'width': '48%', 'display': 'inline-block'}),
            ], style={'marginBottom': 30}),
        
        # Bouton de prédiction
        html.Div([
            html.Button("Prédire", id="predict-btn", n_clicks=0,
                            style={'padding': '10px 25px', 'fontSize': '16px', 'backgroundColor': '#0074D9', 
                             'color': 'white', 'border': 'none', 'borderRadius': '5px', 'cursor': 'pointer'})
        ], style={'textAlign': 'center', 'marginBottom': 30}),
        
        # Message d'information
        html.Div(id="info-message", style={'color': 'red', 'textAlign': 'center', 'marginBottom': 10}),
        
        # Résultat
        html.Div(id="prediction-result", 
                style={'fontSize': '20px', 'padding': '20px', 'textAlign': 'center', 
                      'borderRadius': '5px'})
    ], style={'maxWidth': '900px', 'margin': '0 auto', 'padding': '20px'})
])
    return layout
     #   return page_3()
    #else:
        #return "Page non trouvée"

@app.callback(
    Output('scatter-graph', 'figure'),
    Input('year-range-slider', 'value')
)
def update_scatter(selected_range):
        filtre = Annees_dons_occur[(Annees_dons_occur["Années"] >= selected_range[0]) & (Annees_dons_occur["Années"] <= selected_range[1])]
        fig1 = px.scatter(filtre,x="Années",y="Nombre de dons",color_discrete_sequence=["red"],
                         labels={'x':"Année", 'y':"Nombre de dons de sang enregistrés"},
                         title=f"Evolution des dons de sang {selected_range[0]} à {selected_range[1]}")
        fig1.update_traces(mode="lines+markers")
        return fig1
@app.callback(
    Output('heatmap-graph', 'figure'),
    
    Input('arrondissements-checklist', 'value')
)
def update_heatmap(arround_choisis):
    filtre = tableau_croise.loc[arround_choisis]
    fig2 = px.imshow(filtre,text_auto = True, color_continuous_scale="RdBu",labels=dict(x="Donateur de sang",y="Arrondissement de résidence",
                    color="Frequence"), title="Relation entre arrondissement de résidence et don de sang")
    return fig2
@app.callback(
    Output('sunburst2-graph', 'figure'),
    Input('Eligibilite-Don-avant', 'value')
)
def update_sunburst2(ordre_choisi):
    if ordre_choisi == "CS":

        fig3 = px.sunburst(Base,path=["ÉLIGIBILITÉ AU DON.","A-t-il (elle) déjà donné le sang"],title="Eligibilité-Etat de participation aux campagnes précédentes")
    else:
        fig3 = px.sunburst(Base,path=["A-t-il (elle) déjà donné le sang","ÉLIGIBILITÉ AU DON."],title="Eligibilité-Etat de participation aux campagnes précédentes")
    return fig3
@app.callback(
    Output('box1-graph', 'figure'),
    Input('Eligibilite-Hémoglobine', 'value')
)
def update_box(categorie_choisis):
    filtre = Base[Base["ÉLIGIBILITÉ AU DON."].isin(categorie_choisis)]
    fig4 = px.box(filtre,x="ÉLIGIBILITÉ AU DON.",y="Taux d’hémoglobine",color="ÉLIGIBILITÉ AU DON.",color_discrete_sequence=["#0ABAB5","red","purple"],title="Distribution de l'éligibilité au don de sang en fonction du taux d'hémoglobine")
    fig4.update_yaxes(range=[5,20.5])
    return fig4
    
@app.callback(
    Output('barh1-graph', 'figure'),
    Input('raisons-indisponibilité', 'value')
)
def update_bar(raisons_choisies):
    filtre = Occur_Raison_df[Occur_Raison_df["Raisons d'indisponibilité totale"].isin(raisons_choisies)]
    fig5 = px.bar(filtre, y="Raisons d'indisponibilité totale",x="Occurrences",orientation='h',opacity=0.5)
    fig5.update_traces(marker_line_color='blue',marker_line_width=2)
    return fig5
    # Fonction pour créer la carte
def create_folium_map(selected_arrondissement, selected_quartier):
        # Filtrage des données
    filtered_data = Base_Long_Lat[
        (Base_Long_Lat['Arrondissement de résidence'] == selected_arrondissement) & 
        (Base_Long_Lat['Quartier de Résidence'] == selected_quartier)
    ]
        
        # Création de la carte centrée sur la zone
    m = folium.Map(
        location=[filtered_data['latitude'].mean(), filtered_data['longitude'].mean()], 
        zoom_start=14
    )
        
        # Groupe de marqueurs
    marker_cluster = MarkerCluster().add_to(m)
        
        # Ajout des points pour chaque donneur
    for idx, row in filtered_data.iterrows():
            # Gestion des données manquantes
        def get_value(value):
            return value if pd.notna(value) else "Pas mentionné"
            
            # Texte du popup
        popup_content = f"""
            <b>Date de remplissage :</b> {get_value(row['Date de remplissage de la fiche'])}<br>
            <b>Date de naissance :</b> {get_value(row['Date de naissance'])}<br>
            <b>Niveau d'étude :</b> {get_value(row["Niveau d'etude"])}<br>
            <b>Genre :</b> {get_value(row['Genre'])}<br>
            <b>Taille :</b> {get_value(row['Taille'])} (cm)<br>
            <b>Poids :</b> {get_value(row['Poids'])} (kg)<br>
            <b>Situation Matrimoniale :</b> {get_value(row['Situation Matrimoniale (SM)'])}<br>
            <b>Profession :</b> {get_value(row['Profession'])}<br>
            <b>Arrondissement :</b> {get_value(row['Arrondissement de résidence'])}<br>
            <b>Quartier :</b> {get_value(row['Quartier de Résidence'])}<br>
            <b>Nationalité :</b> {get_value(row['Nationalité'])}<br>
            <b>Religion :</b> {get_value(row['Religion'])}<br>
        """
            
            # Ajout du marqueur sur la carte
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=folium.Popup(popup_content, max_width=300),
        ).add_to(marker_cluster)
        
        # Sauvegarde de la carte
    map_file = f'templates/map_{selected_arrondissement}_{selected_quartier}.html'
    m.save(map_file)
    return map_file
    
    # Mise à jour des quartiers quand on change l'arrondissement
@app.callback(
    Output('quartier-dropdown', 'options'),
    Output('quartier-dropdown', 'value'),
    Input('arrondissement-dropdown', 'value')
)
def update_quartiers(selected_arrondissement):
    quartiers = Base_Long_Lat[
        Base_Long_Lat['Arrondissement de résidence'] == selected_arrondissement
    ]['Quartier de Résidence'].unique()
        
    options = [{'label': q, 'value': q} for q in sorted(quartiers)]
    return options, quartiers[0] if len(quartiers) > 0 else None
    
    # Mise à jour de la carte quand on change les filtres
@app.callback(
    Output('folium-map', 'srcDoc'),
    Input('arrondissement-dropdown', 'value'),
    Input('quartier-dropdown', 'value')
)
def update_map(selected_arrondissement, selected_quartier):
    if selected_arrondissement and selected_quartier:
        map_file = create_folium_map(selected_arrondissement, selected_quartier)
        with open(map_file, 'r', encoding='utf-8') as f:
            return f.read()
    return ""

# Callback pour afficher/masquer les champs "Autre"
@app.callback(
    [Output("religion_autre", "style"), Output("profession_autre", "style")],
    [Input("religion", "value"), Input("profession", "value")]
)
def show_other_fields(religion, profession):
    religion_style = {'width': '100%', 'padding': '8px', 'marginTop': 10} if religion == "Autre" else {'display': 'none'}
    profession_style = {'width': '100%', 'padding': '8px', 'marginTop': 10} if profession == "Autre" else {'display': 'none'}
    return religion_style, profession_style

@app.callback(
    [Output("prediction-result", "children"),
     Output("prediction-result", "style"),
     Output("info-message", "children")],
    [Input("predict-btn", "n_clicks")],
    [State("age", "value"),
     State("taux_hb", "value"),
     State("genre", "value"),
     State("niveau_etude", "value"),
     State("situation_matrimoniale", "value"),
     State("don_sang", "value"),
     State("religion", "value"),
     State("religion_autre", "value"),
     State("profession", "value"),
     State("profession_autre", "value")]
)
def predict(n_clicks, age, taux_hb, genre, niveau_etude, situation_matrimoniale, don_sang, religion, religion_autre, profession, profession_autre):
    default_style = {
        'fontSize': '20px', 
        'padding': '20px', 
        'textAlign': 'center', 
        'borderRadius': '5px',
        'backgroundColor': '#f8f9fa'
    }
    
    if n_clicks == 0:
        return "", default_style, ""
    
    # Vérification des champs obligatoires (tous sauf taux_hb)
    missing_fields = []
    if age is None: missing_fields.append("Âge")
    if genre is None: missing_fields.append("Genre")
    if niveau_etude is None: missing_fields.append("Niveau d'étude")
    if situation_matrimoniale is None: missing_fields.append("Situation matrimoniale")
    if don_sang is None: missing_fields.append("Don de sang")
    if religion is None: missing_fields.append("Religion")
    if profession is None: missing_fields.append("Profession")
    
    if missing_fields:
        return "", default_style, f"Veuillez remplir tous les champs obligatoires: {', '.join(missing_fields)}"
    
    # Gestion du taux d'hémoglobine manquant (seul champ non obligatoire)
    info_message = ""
    if taux_hb is None or taux_hb == "":
        taux_hb = hb_median
        info_message = f"Le taux d'hémoglobine a été remplacé par la médiane ({hb_median})"
    
    # Utiliser la valeur "Autre" si fournie
    final_religion = religion_autre if religion == "Autre" else religion
    final_profession = profession_autre if profession == "Autre" else profession
    
    # Préparation des données pour l'API
    data = {
        "Âge": age,
        "Niveau_d_etude": niveau_etude,
        "Genre": genre,
        "Situation_Matrimoniale": situation_matrimoniale,  
        "Déjà_donné_le_sang": don_sang, 
        "Religion": final_religion,
        "Taux_d_hémoglobine": float(taux_hb) if taux_hb else None,
        "Profession": final_profession
    }
    
    try:
        response = requests.post("http://127.0.0.1:8001/predict", json=data)
        response.raise_for_status()
        result = response.json()
        
        # Détermination de la couleur en fonction du résultat
        if "Éligible" in result:
            result_style = {
                **default_style,
                'backgroundColor': '#d4edda',
                'color': '#155724',
                'border': '1px solid #c3e6cb'
            }
        elif "Définitivement non-eligible" in result:
            result_style = {
                **default_style,
                'backgroundColor': '#f8d7da',
                'color': '#721c24',
                'border': '1px solid #f5c6cb'
            }
        elif "Temporairement Non-eligible" in result:
            result_style = {
                **default_style,
                'backgroundColor': '#fff3cd',
                'color': '#856404',
                'border': '1px solid #ffeeba'
            }
        else:
            result_style = default_style
            
        return f"Résultat : {result}", result_style, info_message
        
    except requests.exceptions.RequestException as e:
        return f"Erreur de connexion à l'API: {str(e)}", {
            **default_style,
            'backgroundColor': '#f8d7da',
            'color': '#721c24'
        }, info_message
    except Exception as e:
        return f"Erreur inattendue: {str(e)}", {
            **default_style,
            'backgroundColor': '#f8d7da',
            'color': '#721c24'
        }, info_message
# Définir les layouts des pages

#def page_3():
 #   return html.Div([
  #      html.H3("Page 3"),
        # Ajouter ici les composants de la page 3 (graphique, table, etc.)
   #     dcc.Graph(figure=px.line(x=[1, 2, 3], y=[4, 5, 6]))
    #])
threading.Timer(1,webbrowser.open_new("http://127.0.0.1:8050/")).start()
# Lancer l'application
if __name__ == '__main__':
    app.run(debug=False)